In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(""), '..')))

In [36]:
from src.Analyzer import Analyzer
from src.DataLoader import DataLoader
import warnings
warnings.simplefilter(action="ignore")

dl = DataLoader()

df = dl.merge_dfs()

Loading Robinhood data
Loading CRSP data
Merging...


In [43]:
import pandas as pd
import numpy as np

def build_returns(group, horizons=[2, 5, 15, 30, 60, 120]):
    group = group.sort_values('date')
    
    # Compute log returns
    group['return_1d'] = np.log(group['prc_adj'] / group['prc_adj'].shift(1))
    
    # Cumulative returns over rolling horizons
    for h in horizons:
        group[f'return_{h}d'] = group['return_1d'].rolling(h, min_periods=1).sum()
        group[f'std_{h}d'] = group['return_1d'].rolling(h, min_periods=h).std()
    
    return group

# Apply the function groupby ticker
df = df.sort_values(['ticker', 'date'])
df = df.groupby('ticker', group_keys=False).apply(build_returns).reset_index(drop=True)

In [44]:
df

,date,ticker,holders,shrcd,vol,prc_adj,shrout_adj,daily_returns,cumulative_returns,mc,...,return_15d,std_15d,return_30d,std_30d,return_60d,std_60d,return_120d,std_120d,return_2d,std_2d
0,2018-05-04,A,587.458333,11.0,1331017.0,67.00,322477000.0,0.009900,0.016402,2.160596e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-07,A,585.900000,11.0,1468735.0,67.39,322477000.0,0.005804,0.022206,2.173173e+10,...,0.005804,NaN,0.005804,NaN,0.005804,NaN,0.005804,NaN,0.005804,NaN
2,2018-05-08,A,577.125000,11.0,1916103.0,67.37,322477000.0,-0.000297,0.021910,2.172528e+10,...,0.005507,NaN,0.005507,NaN,0.005507,NaN,0.005507,NaN,0.005507,0.004314
3,2018-05-09,A,581.208333,11.0,2108506.0,68.13,322477000.0,0.011218,0.033127,2.197036e+10,...,0.016725,NaN,0.016725,NaN,0.016725,NaN,0.016725,NaN,0.010921,0.008142
4,2018-05-10,A,583.041667,11.0,1844868.0,68.86,322477000.0,0.010658,0.043785,2.220577e+10,...,0.027383,NaN,0.027383,NaN,0.027383,NaN,0.027383,NaN,0.021876,0.000396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3157904,2020-08-07,ZYNE,32151.291667,11.0,542213.0,4.22,29255000.0,-0.018780,-0.897151,1.234561e+08,...,0.192803,0.029242,-0.405465,0.129254,0.011919,0.103787,-0.143259,0.083554,0.016727,0.038386
3157905,2020-08-10,ZYNE,32060.833333,11.0,584599.0,4.24,29255000.0,0.004728,-0.892423,1.240412e+08,...,0.171998,0.029091,-0.395986,0.129288,0.050797,0.103690,-0.126134,0.083549,-0.014052,0.016623
3157906,2020-08-11,ZYNE,31939.875000,11.0,449620.0,4.07,29255000.0,-0.040920,-0.933344,1.190679e+08,...,0.128280,0.032060,-0.474294,0.129022,0.014852,0.103828,-0.167054,0.083628,-0.036192,0.032278
3157907,2020-08-12,ZYNE,31851.578947,11.0,980844.0,3.96,29255000.0,-0.027399,-0.960742,1.158498e+08,...,0.065212,0.032383,0.161331,0.041743,-0.210948,0.100565,-0.217065,0.083632,-0.068319,0.009561


In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

df = df.sort_values(by="date")

cols = [
 'holders',
 'shrcd',
 'vol',
 'prc_adj',
 'shrout_adj',
 'cumulative_returns',
 'mc',
 'mc_retail',
 'total_holders',
 'popularity',
 'market_weight',
 'retail_weight',
 'total_holders_change_pct',
 'total_holders_change_diff',
 'return_1d', 
 'return_2d', 
 'return_5d', 
 'return_15d', 
 'return_30d', 
 'return_60d', 
 'return_120d',
 'std_2d', 
 'std_5d', 
 'std_15d', 
 'std_30d', 
 'std_60d', 
 'std_120d'

 ]

# Define input (X) and target (y)
X = df[cols]
y = df['holders_change_pct']

In [46]:
import statsmodels.api as sm
import pandas as pd



# Refit OLS on selected features
X = sm.add_constant(X)  # exclude const
refit_model = sm.OLS(y, X).fit()

# Print p-values
print("\nRefit p-values for selected features:\n")
print(refit_model.summary())






Refit p-values for selected features:

                            OLS Regression Results                            
Dep. Variable:     holders_change_pct   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     6204.
Date:                Fri, 28 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:01:40   Log-Likelihood:             4.0232e+06
No. Observations:             2443105   AIC:                        -8.046e+06
Df Residuals:                 2443078   BIC:                        -8.046e+06
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [47]:
X.corr()

,const,holders,shrcd,vol,prc_adj,shrout_adj,cumulative_returns,mc,mc_retail,total_holders,...,return_15d,return_30d,return_60d,return_120d,std_2d,std_5d,std_15d,std_30d,std_60d,std_120d
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
holders,NaN,1.000000,-0.070403,0.456709,0.066146,0.434903,-0.006263,0.416416,0.317253,0.080107,...,0.020631,0.024499,0.020352,-0.012306,0.038961,0.056402,0.070971,0.083176,0.098824,0.109538
shrcd,NaN,-0.070403,1.000000,-0.035750,0.008283,-0.126068,0.098825,-0.082149,-0.019335,0.003860,...,0.011950,0.021081,0.035578,0.076514,-0.179339,-0.244183,-0.287260,-0.310921,-0.334600,-0.363429
vol,NaN,0.456709,-0.035750,1.000000,0.015299,0.469878,-0.040373,0.273156,0.077773,0.042835,...,0.010984,-0.002572,-0.012252,-0.026220,0.134011,0.142894,0.127121,0.115673,0.107565,0.096069
prc_adj,NaN,0.066146,0.008283,0.015299,1.000000,0.031182,0.183872,0.296836,0.273940,-0.010039,...,0.038710,0.054955,0.078243,0.123552,-0.059684,-0.081540,-0.097427,-0.106312,-0.112228,-0.116104
shrout_adj,NaN,0.434903,-0.126068,0.469878,0.031182,1.000000,0.036335,0.606665,0.096646,-0.002477,...,0.003285,0.005463,0.009222,0.018460,-0.013516,-0.018420,-0.022728,-0.025231,-0.027546,-0.030441
cumulative_returns,NaN,-0.006263,0.098825,-0.040373,0.183872,0.036335,1.000000,0.096917,0.031775,-0.143546,...,0.221964,0.311291,0.428550,0.653962,-0.258198,-0.351180,-0.417867,-0.452681,-0.470501,-0.472428
mc,NaN,0.416416,-0.082149,0.273156,0.296836,0.606665,0.096917,1.000000,0.501556,0.000524,...,0.018878,0.027328,0.039499,0.065068,-0.030897,-0.042623,-0.051074,-0.055735,-0.059117,-0.061333
mc_retail,NaN,0.317253,-0.019335,0.077773,0.273940,0.096646,0.031775,0.501556,1.000000,0.022415,...,0.010420,0.015513,0.019791,0.024657,-0.000178,-0.000505,-0.000728,-0.000901,-0.000167,0.003091
total_holders,NaN,0.080107,0.003860,0.042835,-0.010039,-0.002477,-0.143546,0.000524,0.022415,1.000000,...,0.103874,0.115936,0.088780,-0.155113,0.129285,0.181808,0.233280,0.296909,0.403262,0.477826
